In [27]:
import heapq
import scipy.stats as sts

class Event:
    '''
    Store the properties of one event in the Schedule class defined below. Each
    event has a time at which it needs to run, a function to call when running
    the event, along with the arguments and keyword arguments to pass to that
    function.
    '''
    def __init__(self, timestamp, function, *args, **kwargs):
        self.timestamp = timestamp
        self.function = function
        self.args = args
        self.kwargs = kwargs

    def __lt__(self, other):
        '''
        This overloads the less-than operator in Python. We need it so the
        priority queue knows how to compare two events. We want events with
        earlier (smaller) times to go first.
        '''
        return self.timestamp < other.timestamp

    def run(self, schedule):
        '''
        Run an event by calling the function with its arguments and keyword
        arguments. The first argument to any event function is always the
        schedule in which events are being tracked. The schedule object can be
        used to add new events to the priority queue.
        '''
        self.function(schedule, *self.args, **self.kwargs)


class Schedule:
    '''
    Implement an event schedule using a priority queue. You can add events and
    run the next event.
    
    The `now` attribute contains the time at which the last event was run.
    '''
    
    def __init__(self):
        self.now = 0  # Keep track of the current simulation time
        self.priority_queue = []  # The priority queue of events to run
    
    def add_event_at(self, timestamp, function, *args, **kwargs):
        # Add an event to the schedule at a particular point in time.
        heapq.heappush(
            self.priority_queue,
            Event(timestamp, function, *args, **kwargs))
    
    def add_event_after(self, interval, function, *args, **kwargs):
        # Add an event to the schedule after a specified time interval.
        self.add_event_at(self.now + interval, function, *args, **kwargs)
    
    def next_event_time(self):
        return self.priority_queue[0].timestamp

    def run_next_event(self):
        # Get the next event from the priority queue and run it.
        event = heapq.heappop(self.priority_queue)
        self.now = event.timestamp
        event.run(self)
        
    def __repr__(self):
        return (
            f'Schedule() at time {self.now} ' +
            f'with {len(self.priority_queue)} events in the queue')
    
    def print_events(self):
        print(repr(self))
        for event in sorted(self.priority_queue):
            print(f'   {event.timestamp}: {event.function.__name__}')

In [31]:
import numpy as np

class Queue:
    #this class stores the number of customers in the queue and the number currently being served
    def __init__(self, service_rate):
        self.service_time = 1 / service_rate
        # Initial state: an empty queue and the server is available to serve
        self.number_in_queue = 0
        self.number_being_served = 0
        
    def add_customer(self, schedule):
        self.number_in_queue += 1
        print(
            f'{schedule.now:5.2f}: Add customer to queue.  '
            f'Queue length: {self.number_in_queue}')
        if self.number_being_served < 1: # If no one is being served, this customer can be served immediately
            schedule.add_event_after(0, self.start_serving_customer)
    
    def start_serving_customer(self, schedule):
        # Move the customer from the queue to a server
        self.number_in_queue -= 1
        self.number_being_served += 1
        print(
            f'{schedule.now:5.2f}: Start serving customer. '
            f'Queue length: {self.number_in_queue}')
        # Schedule when the server will be done with the customer
        schedule.add_event_after(
            self.service_time,
            self.finish_serving_customer)
    
    def finish_serving_customer(self, schedule):
        # Remove the customer from the server
        self.number_being_served -= 1
        print(
            f'{schedule.now:5.2f}: Stop serving customer.  '
            f'Queue length: {self.number_in_queue}')
        if self.number_in_queue > 0:
            # There are more people in the queue so serve the next customer
            schedule.add_event_after(0, self.start_serving_customer)
    
    
class BusSystem:
    
    def __init__(self, arrival_rate, service_rate):
        self.queue = Queue(service_rate)
        self.arrival_dist = sts.expon(scale=1/arrival_rate)
    
    def add_customer(self, schedule):
        # Add this customer to the queue
        self.queue.add_customer(schedule)
        # Schedule when to add another customer
        schedule.add_event_after(
            self.arrival_dist.rvs(),
            self.add_customer)

    def run(self, schedule):
        # Schedule when the first customer arrives
        schedule.add_event_after(
            self.arrival_dist.rvs(),
            self.add_customer)
        


In [32]:
def run_simulation(arrival_rate, service_rate, run_until):
    schedule = Schedule()
    bus_system = BusSystem(arrival_rate, service_rate)
    bus_system.run(schedule)
    while schedule.next_event_time() < run_until:
        schedule.run_next_event()
    return bus_system

In [36]:
arrival_rate = 2
service_rate = 1.5
duration = 50
bus_system = run_simulation(arrival_rate, service_rate, duration)
print(f'There are {bus_system.queue.number_in_queue} number of people in the queue')

 1.06: Add customer to queue.  Queue length: 1
 1.06: Start serving customer. Queue length: 0
 1.16: Add customer to queue.  Queue length: 1
 1.73: Stop serving customer.  Queue length: 1
 1.73: Start serving customer. Queue length: 0
 2.34: Add customer to queue.  Queue length: 1
 2.39: Stop serving customer.  Queue length: 1
 2.39: Start serving customer. Queue length: 0
 2.48: Add customer to queue.  Queue length: 1
 3.06: Stop serving customer.  Queue length: 1
 3.06: Start serving customer. Queue length: 0
 3.17: Add customer to queue.  Queue length: 1
 3.57: Add customer to queue.  Queue length: 2
 3.73: Stop serving customer.  Queue length: 2
 3.73: Start serving customer. Queue length: 1
 4.39: Stop serving customer.  Queue length: 1
 4.39: Start serving customer. Queue length: 0
 4.59: Add customer to queue.  Queue length: 1
 5.06: Stop serving customer.  Queue length: 1
 5.06: Start serving customer. Queue length: 0
 5.10: Add customer to queue.  Queue length: 1
 5.45: Add cu

# Reflection

#PythonImplementation: The code implemented a working simulation of a bus system that contains only one bus stop. An appropriate data structure of heap was used. OOP was used to stay organized. 

#CodeReadability: There is no arbitrary number inside code. There are inline code comments to explain the functioning of big blocks of code. The programming style is also consistent. 